In [1]:
import json

def load_ndjson(path):
    data = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                data.append(json.loads(line))
    return data

# Load all splits
train_1 = load_ndjson("train00.json")
train_2 = load_ndjson("train01.json")
train_3 = load_ndjson("train02.json")
train_4 = load_ndjson("train03.json")
dev_data = load_ndjson("valid.json")
test_data = load_ndjson("test.json")

# Merge train splits
train_data = train_1 + train_2 + train_3 + train_4
print(f"Train sentences: {len(train_data)}  | Dev: {len(dev_data)} | Test: {len(test_data)}")

Train sentences: 59924  | Dev: 8528 | Test: 8262


In [2]:
# Optional: quick alignment check
def check_alignment(data, name):
    bad = sum(1 for e in data if len(e["tokens"]) != len(e["tags"]))
    print(f"{name}: {bad} misaligned sentences")

check_alignment(train_data, "train")
check_alignment(dev_data,   "dev")
check_alignment(test_data,  "test")

train: 0 misaligned sentences
dev: 0 misaligned sentences
test: 0 misaligned sentences


In [3]:
# Load the label dictionary (tag string -> index) and build inverse mapping
with open("label.json", encoding="utf-8") as f:
    tag2idx = json.load(f)


tag2idx['<PAD_TAG>'] = len(tag2idx)
idx2tag = {i: t for t, i in tag2idx.items()}
PAD_TAG_IDX = tag2idx['<PAD_TAG>']
num_labels = len(tag2idx)


# Convert dataset entries into (word, tag_str) pairs per sentence
def to_pairs(data):
    sents = []
    for e in data:
        tokens = e["tokens"]
        tags_i = e["tags"]
        tags_s = [idx2tag[int(t)] for t in tags_i]
        sents.append(list(zip(tokens, tags_s)))
    return sents

train_sents = to_pairs(train_data)
dev_sents   = to_pairs(dev_data)
test_sents  = to_pairs(test_data)
print("Sample sentence:", train_sents[0][:10])


Sample sentence: [('People', 'O'), ('start', 'O'), ('their', 'O'), ('own', 'O'), ('businesses', 'O'), ('for', 'O'), ('many', 'O'), ('reasons', 'O'), ('.', 'O')]


# Tokenizer and WordPiece label alignment

In [4]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
MAX_LEN = 128  # adjust based on sentence lengths

def encode_sentence(sent):
    """
    Align labels to first subword of each token.
    Mask other subwords and special/pad tokens with -100.
    """
    tokens = [w for (w, _) in sent]
    tags   = [tag2idx[t] for (_, t) in sent]  # convert tag strings to ids

    enc = tokenizer(tokens,
                    is_split_into_words=True,
                    truncation=True,
                    padding="max_length",
                    max_length=MAX_LEN)

    word_ids = enc.word_ids()
    aligned_labels = []
    prev_word = None
    for wid in word_ids:
        if wid is None:
            aligned_labels.append(-100)          # [CLS], [SEP], padding
        elif wid != prev_word:
            aligned_labels.append(tags[wid])     # first subword gets label
        else:
            aligned_labels.append(-100)          # subsequent subwords ignored in loss
        prev_word = wid

    enc["labels"] = aligned_labels
    return enc

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Dataset class

In [5]:
import torch
from torch.utils.data import Dataset

class NERDataset(Dataset):
    def __init__(self, sentences):
        self.encodings = [encode_sentence(s) for s in sentences]

    def __getitem__(self, idx):
        return {k: torch.tensor(v) for k, v in self.encodings[idx].items()}

    def __len__(self):
        return len(self.encodings)

train_ds = NERDataset(train_sents)
dev_ds   = NERDataset(dev_sents)
test_ds  = NERDataset(test_sents)

# Model and metrics

In [6]:
!pip install transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=8a1eb9883301b48de66f27020910a42091d5a1ad56e2f13a75ab9777778c8ffe
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [7]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00


In [8]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from datasets import load_dataset
from evaluate import load as load_metric

In [9]:
from transformers import BertForTokenClassification

model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=num_labels,
    id2label=idx2tag,    # index -> tag string
    label2id=tag2idx     # tag string -> index
)

from evaluate import load as load_metric
seqeval = load_metric("seqeval")

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids

    true_preds, true_labels = [], []
    for pred, lab in zip(preds, labels):
        pred_tags, lab_tags = [], []
        for p_i, l_i in zip(pred, lab):
            if l_i == -100:
                continue
            pred_tags.append(idx2tag[p_i])
            lab_tags.append(idx2tag[l_i])
        true_preds.append(pred_tags)
        true_labels.append(lab_tags)

    res = seqeval.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": res["overall_precision"],
        "recall": res["overall_recall"],
        "f1": res["overall_f1"],
        "accuracy": res["overall_accuracy"],
    }

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Training arguments

In [10]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="bert-pos-output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="logs",
)

# Train

In [11]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-1867442277.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: slimaniaymen113 (slimaniaymen113-university-of-michigan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.304300
1000,0.112700
1500,0.091600
2000,0.084800
2500,0.080200
3000,0.080200
3500,0.071600
4000,0.058100
4500,0.054200
5000,0.051600


TrainOutput(global_step=11238, training_loss=0.06481380822459538, metrics={'train_runtime': 4273.8913, 'train_samples_per_second': 42.063, 'train_steps_per_second': 2.629, 'total_flos': 1.1747282326566912e+16, 'train_loss': 0.06481380822459538, 'epoch': 3.0})

# Evaluate on the test set

In [12]:
test_results = trainer.evaluate(test_ds)
print("Test results:", test_results)

Test results: {'eval_loss': 0.07511512190103531, 'eval_precision': 0.8742743263148774, 'eval_recall': 0.8964904486894714, 'eval_f1': 0.8852430250921214, 'eval_accuracy': 0.9814766296373926, 'eval_runtime': 64.2259, 'eval_samples_per_second': 128.64, 'eval_steps_per_second': 8.05, 'epoch': 3.0}


In [14]:
from seqeval.metrics import classification_report
from sklearn.metrics import accuracy_score

pred_output = trainer.predict(test_ds)
preds = pred_output.predictions.argmax(-1)
labels = pred_output.label_ids

true_seq, pred_seq = [], []
for pred, lab in zip(preds, labels):
    t_tags, p_tags = [], []
    for p_i, l_i in zip(pred, lab):
        if l_i == -100:
            continue
        t_tags.append(idx2tag[l_i])
        p_tags.append(idx2tag[p_i])
    true_seq.append(t_tags)
    pred_seq.append(p_tags)
flat_true = [tag for seq in true_seq for tag in seq]
flat_pred = [tag for seq in pred_seq for tag in seq]
acc = accuracy_score(flat_true, flat_pred)
print("Token-level accuracy:", round(acc, 3))
print(classification_report(true_seq, pred_seq, digits=3))

Token-level accuracy: 0.981
              precision    recall  f1-score   support

    CARDINAL      0.851     0.860     0.855       935
        DATE      0.831     0.886     0.858      1601
       EVENT      0.641     0.651     0.646        63
         FAC      0.715     0.763     0.738       135
         GPE      0.964     0.940     0.952      2240
    LANGUAGE      0.824     0.636     0.718        22
         LAW      0.658     0.625     0.641        40
         LOC      0.687     0.810     0.744       179
       MONEY      0.852     0.898     0.874       314
        NORP      0.877     0.902     0.890       841
     ORDINAL      0.801     0.908     0.851       195
         ORG      0.888     0.900     0.894      1795
     PERCENT      0.901     0.911     0.906       348
      PERSON      0.948     0.952     0.950      1988
     PRODUCT      0.544     0.737     0.626        76
    QUANTITY      0.752     0.810     0.780       105
        TIME      0.613     0.679     0.644       212

In [15]:
from sklearn.metrics import accuracy_score

flat_true = [tag for seq in true_seq for tag in seq]
flat_pred = [tag for seq in pred_seq for tag in seq]

acc = accuracy_score(flat_true, flat_pred)
print("Token-level accuracy:", round(acc, 3))


Token-level accuracy: 0.981


# BERT ON THE MOVIE DATASET

In [16]:
import json

# Load the JSON file
with open("movies.json", "r", encoding="utf-8") as f:
    script_data = json.load(f)

In [17]:
print("First few entries in script_data:", script_data[:3])

First few entries in script_data: [{'Name': 'I am Sam', 'Script': '"I AM SAM"\r\n\r\n                                      Screenplay by\r\n\r\n                                     Kristine Johnson\r\n\r\n                                      Jessie Nelson\r\n\r\n                                           2001\r\n\r\n                                      SHOOTING DRAFT\r\n\r\n\r\n\r\n            INT. STARBUCKS - 7:30 A.M.\r\n\r\n            We\'re watching a pair of hands arrange white sugar packets,\r\n            blue Equal packets, and pink Sweet and Low into small\r\n            containers.  With precision and lightning speed, the mixed up\r\n            colors and crumpled packets are transformed into neat little\r\n            color-coded rows.  Wait, this container has three Equals and\r\n            four Sweet \'n\' lows.  The hand quickly plucks the mutant\r\n            Sweet \'n\' Low.  There.  Symmetry.\r\n\r\n            We move up those hands and meet SAM DAWSON as he sur

In [18]:
import re

def is_valid_line(line):
    line = line.strip()
    return not line.isupper() and len(line.split()) > 3

def simple_tokenize(text):
    return re.findall(r"\w[\w']*", text)

script_sentences = []
for entry in script_data:
    if isinstance(entry, dict) and "Script" in entry:
        lines = entry["Script"].splitlines()
        for line in lines:
            if is_valid_line(line):
                tokens = simple_tokenize(line)
                if tokens:
                    script_sentences.append(tokens)

In [19]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
MAX_LEN = 128  # match your training setup

def encode_sentence(sent):
    dummy_tags = ["O"] * len(sent)
    tag_ids = [tag2idx[t] for t in dummy_tags]

    enc = tokenizer(sent,
                    is_split_into_words=True,
                    truncation=True,
                    padding="max_length",
                    max_length=MAX_LEN,
                    return_tensors="tf")  # TensorFlow model

    word_ids = enc.word_ids()
    aligned_labels = []
    prev_word = None
    for i, word_id in enumerate(word_ids):
        if word_id is None:
            aligned_labels.append(-100)
        elif word_id != prev_word:
            aligned_labels.append(tag_ids[word_id])
        else:
            aligned_labels.append(-100)
        prev_word = word_id

    enc["labels"] = aligned_labels
    return enc

In [28]:
tokens_decoded = tokenizer.convert_ids_to_tokens(enc["input_ids"][0])
word_ids = enc.word_ids()

prev_word_id = None
for idx, word_id in enumerate(word_ids):
    if word_id is None or word_id == prev_word_id:
        continue
    print(f"{word_id}: {tokens_decoded[idx]}")
    prev_word_id = word_id



0: We
1: watching
2: a
3: pair
4: of
5: hands
6: arrange
7: white
8: sugar
9: packets
